In [ ]:
import sys
import os
sys.path.append('/home/anilbs/OSS/Production/GitRipper')
from tqdm import tqdm
import json
from GitRipper.Ripper import collect
import pandas as pd

In [ ]:
with open("/home/anilbs/OSS/api_keys.json") as f:
    data = json.load(f)

In [ ]:
keys = data['keys']
keys_list = [(k.split(":")[0], k.split(":")[1]) for k in keys]

In [ ]:
collector = collect(keys_list)

In [ ]:
columns = ['Package', 'Slug', 'URL', 'oid', 'messageHeadline', 'author_name', 'author_email',
       'author_user_login', 'author_user_location', 'author_user_company',
       'author_user_pronouns', 'author_user_bio', 'author_user_websiteUrl',
       'author_user_twitterUsername', 'author_date', 'additions', 'deletions']

In [ ]:
#check if the empty csv exists
# if not, create one
# if yes, read the csv and get the list of packages already collected
if os.path.exists('/home/anilbs/OSS/CRAN_GH_Data.csv'):
       df = pd.read_csv('/home/anilbs/OSS/CRAN_GH_Data.csv')
       packages = df['Package'].unique()
# Collecting all the commits information for the given repository
else:
       # Creating an empty csv with the required columns
       df = pd.DataFrame(columns=columns)
       df.to_csv('/home/anilbs/OSS/CRAN_GH_Data.csv', index=False)
       packages = []

In [ ]:
df_cran_slugs = pd.read_csv("/home/anilbs/OSS/Data/cran_github_all_slugs.csv")[['Package', 'Slugs', 'URL']]

In [ ]:
df_cran_slugs.head()

In [ ]:
#select a random row
row = df_cran_slugs.iloc[3]
s = row.Slugs
reponame = s.split("/")[1]
ownername = s.split("/")[0]

In [ ]:
for index, row in tqdm(df_cran_slugs.iterrows()):
    s = row.Slugs
    p = row.Package
    u = row.URL
    if p in packages:
        continue
    try:
        reponame = s.split("/")[1]
        ownername = s.split("/")[0]
    except Exception as e:
        print("Error in parsing slug ", s, "\nMessage: ", e)
        continue
    try:
        df_out = collector.collectCommits(ownername, reponame)
        #Add package name, slug and url to the dataframe
        df_out['Package'] = p
        df_out['Slug'] = s
        df_out['URL'] = u
        #Make sure that package, slug and url are the first three columns
        df_out = df_out[columns]
        df_out.to_csv('./CRAN_GH_Data.csv', mode='a', header=False, index=False)
    except Exception as e:
        print("Error in collecting commits for ", reponame, ownername, "\nMessage: ", e)